**Name :- Anshuman Patel**


**Batch :-  2**

In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 250
l = 17
num_filter = 24
compression = 0.5
dropout_rate = 0.0

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1),
              metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(rotation_range=15,
                               width_shift_range=5./32,
                               height_shift_range=5./32,
                               horizontal_flip=True)

generator.fit(x_train, seed=0)

In [0]:
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) // batch_size,
                     epochs=200,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/200
781/781 [==============================] - 100s 128ms/step - loss: 1.7206 - acc: 0.3764 - val_loss: 1.6422 - val_acc: 0.4412
Epoch 2/200
781/781 [==============================] - 85s 109ms/step - loss: 1.2847 - acc: 0.5356 - val_loss: 1.4396 - val_acc: 0.5152
Epoch 3/200
781/781 [==============================] - 85s 109ms/step - loss: 1.0920 - acc: 0.6096 - val_loss: 1.1377 - val_acc: 0.6041
Epoch 4/200
781/781 [==============================] - 84s 107ms/step - loss: 0.9584 - acc: 0.6591 - val_loss: 0.9352 - val_acc: 0.6669
Epoch 5/200
781/781 [==============================] - 83s 107ms/step - loss: 0.8682 - acc: 0.6941 - val_loss: 1.0008 - val_acc: 0.6597
Epoch 6/200
781/781 [==============================] - 83s 106ms/step - loss: 0.8015 - acc: 0.7185 - val_loss: 0.8480 - val_acc: 0.7110
Epoch 7/200
781/781 [==============================] - 85s 108ms/step - loss: 0.7387 - acc: 0.7403 - val_loss: 0.9749 - val_acc: 0.6708
Epoch 8/200
781/781 [==========================

Epoch 61/200
781/781 [==============================] - 82s 105ms/step - loss: 0.2103 - acc: 0.9263 - val_loss: 0.3821 - val_acc: 0.8803
Epoch 62/200
781/781 [==============================] - 81s 103ms/step - loss: 0.2110 - acc: 0.9253 - val_loss: 0.4135 - val_acc: 0.8733
Epoch 63/200
781/781 [==============================] - 80s 102ms/step - loss: 0.2074 - acc: 0.9253 - val_loss: 0.3451 - val_acc: 0.8895
Epoch 64/200
781/781 [==============================] - 80s 102ms/step - loss: 0.2063 - acc: 0.9284 - val_loss: 0.3743 - val_acc: 0.8816
Epoch 65/200
781/781 [==============================] - 81s 103ms/step - loss: 0.1999 - acc: 0.9283 - val_loss: 0.4049 - val_acc: 0.8807
Epoch 66/200
781/781 [==============================] - 79s 102ms/step - loss: 0.1949 - acc: 0.9316 - val_loss: 0.3924 - val_acc: 0.8802
Epoch 67/200
781/781 [==============================] - 79s 102ms/step - loss: 0.1973 - acc: 0.9291 - val_loss: 0.3942 - val_acc: 0.8783
Epoch 68/200
781/781 [===================

781/781 [==============================] - 79s 101ms/step - loss: 0.1175 - acc: 0.9591 - val_loss: 0.3912 - val_acc: 0.8945
Epoch 121/200
781/781 [==============================] - 79s 101ms/step - loss: 0.1136 - acc: 0.9590 - val_loss: 0.4448 - val_acc: 0.8864
Epoch 122/200
781/781 [==============================] - 79s 102ms/step - loss: 0.1110 - acc: 0.9599 - val_loss: 0.4071 - val_acc: 0.8952
Epoch 123/200
781/781 [==============================] - 80s 103ms/step - loss: 0.1154 - acc: 0.9593 - val_loss: 0.3516 - val_acc: 0.9042
Epoch 124/200
781/781 [==============================] - 79s 101ms/step - loss: 0.1127 - acc: 0.9596 - val_loss: 0.4864 - val_acc: 0.8720
Epoch 125/200
781/781 [==============================] - 79s 101ms/step - loss: 0.1114 - acc: 0.9604 - val_loss: 0.4542 - val_acc: 0.8836
Epoch 126/200
781/781 [==============================] - 79s 101ms/step - loss: 0.1133 - acc: 0.9596 - val_loss: 0.4048 - val_acc: 0.8924
Epoch 127/200
781/781 [=========================

781/781 [==============================] - 79s 101ms/step - loss: 0.0780 - acc: 0.9725 - val_loss: 0.4512 - val_acc: 0.8941
Epoch 180/200
781/781 [==============================] - 79s 101ms/step - loss: 0.0744 - acc: 0.9742 - val_loss: 0.4470 - val_acc: 0.8957
Epoch 181/200
781/781 [==============================] - 79s 101ms/step - loss: 0.0730 - acc: 0.9740 - val_loss: 0.4462 - val_acc: 0.8927
Epoch 182/200
781/781 [==============================] - 80s 103ms/step - loss: 0.0742 - acc: 0.9738 - val_loss: 0.4425 - val_acc: 0.8952
Epoch 183/200
781/781 [==============================] - 79s 101ms/step - loss: 0.0725 - acc: 0.9744 - val_loss: 0.4442 - val_acc: 0.8953
Epoch 184/200
781/781 [==============================] - 79s 101ms/step - loss: 0.0727 - acc: 0.9738 - val_loss: 0.4136 - val_acc: 0.8961
Epoch 185/200
781/781 [==============================] - 79s 101ms/step - loss: 0.0709 - acc: 0.9751 - val_loss: 0.4485 - val_acc: 0.8958
Epoch 186/200
781/781 [=========================

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 71s 1ms/step - loss: 0.0163 - acc: 0.9950 - val_loss: 0.3131 - val_acc: 0.9215
Epoch 2/50
50000/50000 [==============================] - 71s 1ms/step - loss: 0.0057 - acc: 0.9989 - val_loss: 0.3155 - val_acc: 0.9265
Epoch 3/50
50000/50000 [==============================] - 70s 1ms/step - loss: 0.0031 - acc: 0.9996 - val_loss: 0.3289 - val_acc: 0.9252
Epoch 4/50
50000/50000 [==============================] - 70s 1ms/step - loss: 0.0020 - acc: 0.9999 - val_loss: 0.3339 - val_acc: 0.9267
Epoch 5/50
50000/50000 [==============================] - 70s 1ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.3375 - val_acc: 0.9274
Epoch 6/50
50000/50000 [==============================] - 71s 1ms/step - loss: 0.0013 - acc: 0.9999 - val_loss: 0.3461 - val_acc: 0.9292
Epoch 7/50
50000/50000 [==============================] - 70s 1ms/step - loss: 0.0014 - acc: 0.9999 - val_loss: 0.3433 - val_acc

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 4s 430us/step
Test loss: 0.43880280739217997
Test accuracy: 0.9241


The max result obtained was **93.06%** validation accuracy and **0.3904** validation loss at **234th epoch** ( 200 + 34 ).